In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta
#from datetime import timedelta  ## use this to increment targ by 1 year!!!!
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
#import statsmodels.api as sm
#import statsmodels.formula.api as smf
#from patsy import dmatrices
#import random

import sys
import os
os.chdir('C:/BankRisk')

In [2]:
import time
from time import gmtime, strftime
print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

2019-05-24 18:17:24


In [3]:
dm = pd.read_csv("ladder_durmv_allmat_longhist_201903.csv",header=[0,1],index_col=0,parse_dates=True) ## NOTE THIS COMMAND
dm.head(2)

dv12                                                    \
                  ff        mv      mvpc       dur       cpn       mv3   
1965-12-01  4.380000  9.872790  1.008227  0.534467  5.493205  4.784516   
1966-01-01  4.481389  9.288667  1.007741  0.534508  5.463205  4.955346   

                               dv24       ... dv300                 dv360  \
             roll12m    roll3m   mv mvpc  ...   mv3 roll300m roll3m    mv   
1965-12-01  4.854038  4.998342  NaN  NaN  ...   NaN      NaN    NaN   NaN   
1966-01-01  4.916538  5.187601  NaN  NaN  ...   NaN      NaN    NaN   NaN   

                                             
           mvpc dur cpn mv3 roll360m roll3m  
1965-12-01  NaN NaN NaN NaN      NaN    NaN  
1966-01-01  NaN NaN NaN NaN      NaN    NaN  

[2 rows x 106 columns]

In [4]:
dmq = dm.resample("Q").mean()
#dmq = dmq.dropna(how='any')
dmq.head(2)

dv12                                                   \
                  ff       mv      mvpc       dur       cpn       mv3   
1965-12-31  4.380000  9.87279  1.008227  0.534467  5.493205  4.784516   
1966-03-31  4.619954  9.60940  1.008008  0.534447  5.559872  5.331311   

                               dv24       ... dv300                 dv360  \
             roll12m    roll3m   mv mvpc  ...   mv3 roll300m roll3m    mv   
1965-12-31  4.854038  4.998342  NaN  NaN  ...   NaN      NaN    NaN   NaN   
1966-03-31  4.986816  5.337432  NaN  NaN  ...   NaN      NaN    NaN   NaN   

                                             
           mvpc dur cpn mv3 roll360m roll3m  
1965-12-31  NaN NaN NaN NaN      NaN    NaN  
1966-03-31  NaN NaN NaN NaN      NaN    NaN  

[2 rows x 106 columns]

In [5]:
dmq120=pd.DataFrame(dmq['dv120']['roll120m'])
dmq144 = dmq['dv144']['roll144m']
dmq180 = dmq['dv180']['roll180m']
dmq240 = dmq['dv240']['roll240m']
dmq300 = dmq['dv300']['roll300m']
dmq360 = dmq['dv360']['roll360m']
dmx = pd.concat([dmq120,dmq144,dmq180,dmq240,dmq300,dmq360],axis=1)

dmx.tail(2)

roll120m  roll144m  roll180m  roll240m  roll300m  roll360m
2018-12-31  2.572791  3.023983  3.505313  4.238823  4.867810  5.532837
2019-03-31  2.568955  2.974455  3.472789  4.201589  4.830515  5.476720

In [6]:
## get libor-swap history:
ls = pd.read_csv("LibSwap_LongHist_30y_201903.csv",index_col=0)
ls.index = pd.to_datetime(ls.index)
#ls = ls[:'2014']
lsq=ls['1965':].resample('Q').mean()
## prepare to build ladders:
ld = lsq[['F1d','Prime']]
ld['DDA'] = 0.
mlst = [1,3,6,12,24,36,48,60,84,120,360]  # maturity list to be used
lcol = list(ls.columns)[1:-2]
lcol.remove('L12m')
mdct = dict(zip(mlst,lcol))
len(lsq), len(ld) # 217, 217

(217, 217)

In [7]:
mlst

[1, 3, 6, 12, 24, 36, 48, 60, 84, 120, 360]

In [8]:
## alternative code to process rolling averages:
ldm = ls[['F1d','Prime']]
m1 = ls[mdct[mlst[0]]] * 365/360
m3 = ls[mdct[mlst[1]]].rolling(mlst[1],min_periods=mlst[1]).mean() * 365/360
m6 = ls[mdct[mlst[2]]].rolling(mlst[2],min_periods=mlst[2]).mean() * 365/360
m12 = ls[mdct[mlst[3]]].rolling(mlst[3],min_periods=mlst[3]).mean()
m24 = ls[mdct[mlst[4]]].rolling(mlst[4],min_periods=mlst[4]).mean()
m36 = ls[mdct[mlst[5]]].rolling(mlst[5],min_periods=mlst[5]).mean()
m48 = ls[mdct[mlst[6]]].rolling(mlst[6],min_periods=mlst[6]).mean()
m60 = ls[mdct[mlst[7]]].rolling(mlst[7],min_periods=mlst[7]).mean()
m84 = ls[mdct[mlst[8]]].rolling(mlst[8],min_periods=mlst[8]).mean()
m120 = ls[mdct[mlst[9]]].rolling(mlst[9],min_periods=mlst[9]).mean()
m360 = ls[mdct[mlst[10]]].rolling(mlst[10],min_periods=mlst[10]).mean()

ldc = pd.concat([ldm,m1,m3,m6,m12,m24,m36,m48,m60,m84,m120,m360],axis=1)
ldc['F1d'] = ldc['F1d'] * 365/360 
ldc['Prime'] = ldc['Prime'] * 365/360 
#ldc.dropna(how='any',inplace=True)
ldx=ldc.resample('Q').mean()
#ldl=ldc.resample('Q',how='last')  # do not use
ldx.tail(4)

F1d     Prime       L1m       L3m       L6m       S1y  \
Date                                                                     
2018-06-30  1.760787  4.863287  1.997849  2.283632  2.205926  1.886432   
2018-09-30  1.950046  5.079583  2.138821  2.366875  2.501744  2.184707   
2018-12-31  2.250833  5.356713  2.384097  2.533483  2.647178  2.484374   
2019-03-31  2.436713  5.576389  2.532715  2.760413  2.797971  2.669171   

                 S2y       S3y       S4y       S5y       S7y      S10y  \
Date                                                                     
2018-06-30  1.665925  1.567874  1.636014  1.743316  1.910427  2.638290   
2018-09-30  1.904300  1.706647  1.726011  1.813941  1.933648  2.596897   
2018-12-31  2.136647  1.864204  1.820730  1.886135  1.978817  2.572791   
2019-03-31  2.294712  1.993449  1.897902  1.937241  2.016448  2.568955   

                S30y  
Date                  
2018-06-30  5.644563  
2018-09-30  5.587019  
2018-12-31  5.532837  
2019-03-31  5.476720

In [9]:
ldx = ldx['1965-12-31':]
len(ldx),len(dmx) # 214, 214

(214, 214)

In [10]:
ldx.head(2)

F1d     Prime       L1m       L3m       L6m       S1y  S2y  \
Date                                                                          
1965-12-31  4.224537  4.704444  4.611440  4.866537  4.880055  4.854038  NaN   
1966-03-31  4.619954  5.187731  5.033893  5.337432  5.256321  4.986816  NaN   

            S3y  S4y  S5y  S7y  S10y  S30y  
Date                                        
1965-12-31  NaN  NaN  NaN  NaN   NaN   NaN  
1966-03-31  NaN  NaN  NaN  NaN   NaN   NaN

In [11]:
dmx.head(2)

roll120m  roll144m  roll180m  roll240m  roll300m  roll360m
1965-12-31       NaN       NaN       NaN       NaN       NaN       NaN
1966-03-31       NaN       NaN       NaN       NaN       NaN       NaN

In [12]:
ldx[['S12y','S15y','S20y','S25y']] =dmx[['roll144m','roll180m','roll240m','roll300m']]

In [13]:
ldx = ldx[['F1d','L1m','L3m','L6m','S1y','S2y','S3y','S4y','S5y','S7y','S10y','S12y','S15y','S20y','S25y','S30y','Prime']]
ldx['DDA'] = 0
ldx.tail(2)

F1d       L1m       L3m       L6m       S1y       S2y  \
Date                                                                     
2018-12-31  2.250833  2.384097  2.533483  2.647178  2.484374  2.136647   
2019-03-31  2.436713  2.532715  2.760413  2.797971  2.669171  2.294712   

                 S3y       S4y       S5y       S7y      S10y      S12y  \
Date                                                                     
2018-12-31  1.864204  1.820730  1.886135  1.978817  2.572791  3.023983   
2019-03-31  1.993449  1.897902  1.937241  2.016448  2.568955  2.974455   

                S15y      S20y      S25y      S30y     Prime  DDA  
Date                                                               
2018-12-31  3.505313  4.238823  4.867810  5.532837  5.356713    0  
2019-03-31  3.472789  4.201589  4.830515  5.476720  5.576389    0

In [14]:
rlst = ['r'+str(x) for x in mlst+[144,180,240,300]]
rdct = dict(zip(lcol+['S12y','S15y','S20y','S25y'],rlst))
if 'L1m' in ldx.columns: ldx.rename(columns=rdct,inplace=True)
if 'F1d' in ldx.columns: ldx.rename(columns={'F1d':'r0'},inplace=True)
ldx = ldx[['r0','r1','r3','r6','r12','r24','r36','r48','r60','r84','r120','r144','r180','r240','r300','r360','Prime','DDA']]
ldx.tail()
#rdct

r0        r1        r3        r6       r12       r24  \
Date                                                                     
2018-03-31  1.466759  1.676986  1.775693  1.771696  1.614220  1.447438   
2018-06-30  1.760787  1.997849  2.283632  2.205926  1.886432  1.665925   
2018-09-30  1.950046  2.138821  2.366875  2.501744  2.184707  1.904300   
2018-12-31  2.250833  2.384097  2.533483  2.647178  2.484374  2.136647   
2019-03-31  2.436713  2.532715  2.760413  2.797971  2.669171  2.294712   

                 r36       r48       r60       r84      r120      r144  \
Date                                                                     
2018-03-31  1.437075  1.545873  1.651651  1.910147  2.674767  3.175388   
2018-06-30  1.567874  1.636014  1.743316  1.910427  2.638290  3.122483   
2018-09-30  1.706647  1.726011  1.813941  1.933648  2.596897  3.068257   
2018-12-31  1.864204  1.820730  1.886135  1.978817  2.572791  3.023983   
2019-03-31  1.993449  1.897902  1.937241  2.016448  2.568955  2.974455   

                r180      r240      r300      r360     Prime  DDA  
Date                                                               
2018-03-31  3.588781  4.352785  4.978104  5.701569  4.589537    0  
2018-06-30  3.563678  4.311299  4.937103  5.644563  4.863287    0  
2018-09-30  3.536476  4.272151  4.899590  5.587019  5.079583    0  
2018-12-31  3.505313  4.238823  4.867810  5.532837  5.356713    0  
2019-03-31  3.472789  4.201589  4.830515  5.476720  5.576389    0

In [15]:
mzlst = [0,1, 3, 6, 12, 24, 36, 48, 60, 84, 120]
rzlst = ['r0','r1','r3','r6','r12','r24','r36','r48','r60','r84','r120','Prime','DDA']
#if 0 not in mzlst: mzlst = [0] + mlst
#if 'r0' not in rzlst: rzlst = ['r0'] + rlst
ldxall = ldx['2001':].copy()  # keep ldxall as a reference table (for multiple runs)
ldxall.head(2)

r0        r1        r3        r6      r12       r24  \
Date                                                                    
2001-03-31  5.671019  5.584845  5.925261  6.289939  6.58615  6.441290   
2001-06-30  4.386759  4.350878  4.636757  5.156984  5.97003  6.360413   

                 r36       r48       r60       r84      r120      r144  \
Date                                                                     
2001-03-31  6.103628  6.175796  6.266308  6.514782  6.895917  7.321731   
2001-06-30  6.102206  6.095638  6.223161  6.481845  6.828667  7.247147   

                r180      r240      r300      r360     Prime  DDA  
Date                                                               
2001-03-31  7.729644  9.055592  9.238483  9.023238  8.743102    0  
2001-06-30  7.689809  8.956643  9.214836  9.019233  7.441944    0

In [16]:
mzlst

[0, 1, 3, 6, 12, 24, 36, 48, 60, 84, 120]

In [17]:
rzlst

['r0',
 'r1',
 'r3',
 'r6',
 'r12',
 'r24',
 'r36',
 'r48',
 'r60',
 'r84',
 'r120',
 'Prime',
 'DDA']

In [18]:
ldxall.tail(2)

r0        r1        r3        r6       r12       r24  \
Date                                                                     
2018-12-31  2.250833  2.384097  2.533483  2.647178  2.484374  2.136647   
2019-03-31  2.436713  2.532715  2.760413  2.797971  2.669171  2.294712   

                 r36       r48       r60       r84      r120      r144  \
Date                                                                     
2018-12-31  1.864204  1.820730  1.886135  1.978817  2.572791  3.023983   
2019-03-31  1.993449  1.897902  1.937241  2.016448  2.568955  2.974455   

                r180      r240      r300      r360     Prime  DDA  
Date                                                               
2018-12-31  3.505313  4.238823  4.867810  5.532837  5.356713    0  
2019-03-31  3.472789  4.201589  4.830515  5.476720  5.576389    0

In [19]:
grp='TestProd2'

In [20]:
## select time period: 2001-2014, 2001-2007, 2008-2014 (14y, 7y, 7y)
#r0 = '2001_2014'
#r0 = '2001_2007'
r0 = '2001_2010'  # use this for book
#r0 = '2005_2014'
ra = int(r0[0:4])
rb = int(r0[-4:])
r0, ra, rb

('2001_2010', 2001, 2010)

In [21]:
ldx = ldxall[str(ra):str(rb)]
ldx['r1.5'] = ldx['r1']*.5
ldx['r3.5'] = ldx['r3']*.5
ldx['r6.5'] = ldx['r6']*.5
ldx['r12.5'] = ldx['r12']*.5
ldx['r1.5_1'] = ldx['r1.5'].shift(1)
ldx['r3.5_1'] = ldx['r3.5'].shift(1)
ldx['r6.5_1'] = ldx['r6.5'].shift(1)
ldx['r12.5_1'] = ldx['r12.5'].shift(1)

In [22]:
ldx.head(2)

r0        r1        r3        r6      r12       r24  \
Date                                                                    
2001-03-31  5.671019  5.584845  5.925261  6.289939  6.58615  6.441290   
2001-06-30  4.386759  4.350878  4.636757  5.156984  5.97003  6.360413   

                 r36       r48       r60       r84  ...     Prime  DDA  \
Date                                                ...                  
2001-03-31  6.103628  6.175796  6.266308  6.514782  ...  8.743102    0   
2001-06-30  6.102206  6.095638  6.223161  6.481845  ...  7.441944    0   

                r1.5      r3.5      r6.5     r12.5    r1.5_1    r3.5_1  \
Date                                                                     
2001-03-31  2.792422  2.962631  3.144970  3.293075       NaN       NaN   
2001-06-30  2.175439  2.318379  2.578492  2.985015  2.792422  2.962631   

             r6.5_1   r12.5_1  
Date                           
2001-03-31      NaN       NaN  
2001-06-30  3.14497  3.293075  

[2 rows x 26 columns]

In [23]:
ldx.tail(2)

r0        r1        r3        r6       r12       r24  \
Date                                                                     
2010-09-30  0.189259  0.292869  0.459604  0.600520  0.593889  1.486944   
2010-12-31  0.189259  0.260921  0.302902  0.574779  0.550556  1.222222   

                 r36       r48       r60       r84  ...     Prime  DDA  \
Date                                                ...                  
2010-09-30  2.568241  3.456181  3.955611  4.241468  ...  3.295139    0   
2010-12-31  2.265370  3.216111  3.800000  4.170159  ...  3.295139    0   

                r1.5      r3.5     r6.5     r12.5    r1.5_1    r3.5_1  \
Date                                                                    
2010-09-30  0.146434  0.229802  0.30026  0.296944  0.159510  0.178901   
2010-12-31  0.130460  0.151451  0.28739  0.275278  0.146434  0.229802   

              r6.5_1   r12.5_1  
Date                            
2010-09-30  0.239778  0.322361  
2010-12-31  0.300260  0.296944  

[2 rows x 26 columns]

In [24]:

y9col=['r1.5','r1.5_1','r3.5','r3.5_1','r6.5','r6.5_1','r12.5','r12.5_1','DDA','r6','r12']

y9col

['r1.5',
 'r1.5_1',
 'r3.5',
 'r3.5_1',
 'r6.5',
 'r6.5_1',
 'r12.5',
 'r12.5_1',
 'DDA',
 'r6',
 'r12']

In [25]:
mx = np.arange(0,1.05,.05)  # mixes to analyze
yrcol=['grp','prod','mt1','mt2','mt3','mx1','mx2','dur','spd','sd']
yr = pd.DataFrame(columns=yrcol)

In [26]:
mzlst

[0, 1, 3, 6, 12, 24, 36, 48, 60, 84, 120]

In [27]:
rzlst

['r0',
 'r1',
 'r3',
 'r6',
 'r12',
 'r24',
 'r36',
 'r48',
 'r60',
 'r84',
 'r120',
 'Prime',
 'DDA']

In [28]:
d20y = pd.read_csv('durladder_last20y_adjmnts.csv',index_col=0)
d20y.index.names = ['mnth']
d20y.reset_index(drop=False,inplace=True)
d20dct = dict(zip(d20y['mnth'],d20y['dur']))
d20dct[0] = 0
d20dct[1] = 1/24
d20dct[3] = 3/24
d20dct[6] = 6/24
d20dct

{12: 0.5372372286402183,
 24: 1.0202310529246352,
 36: 1.4886787413076978,
 48: 1.9409599189501283,
 60: 2.3760869494703982,
 72: 2.7937928221701878,
 84: 3.1937051845458275,
 96: 3.5757229037740736,
 108: 3.939583317616629,
 120: 4.28872156228373,
 144: 4.949200080085927,
 180: 5.838246756344285,
 240: 7.075555871821653,
 300: 8.089358858424841,
 360: 8.956140867299661,
 0: 0,
 1: 0.041666666666666664,
 3: 0.125,
 6: 0.25}

In [29]:
## WARNING:  This cell takes about 3 hours to process!!
## process set of products in ycol:

for prd in y9col:
  ldx['prd'] = ldx[prd]
  print(prd+' started at:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
#  for i in [0,1]:
  for i in np.arange(0,len(mzlst)-2):
    mt1 = rzlst[i]
    for j in np.arange(i+1,len(mzlst)-1):
      mt2 = rzlst[j]
      for k in np.arange(j+1,len(mzlst)):
        mt3 = rzlst[k]
        for m1 in mx:
          for m2 in mx:
            if (m1==0)&(m2==0): sd=99999
            if (m1 + m2) <= 1:
              ldx['spd'] = ldx.apply(lambda x: x['prd'] - (m1*x[mt1] + m2*x[mt2] + (1-m1-m2)*x[mt3]),axis=1)
              if ldx['spd'].std() < sd:
                sd = ldx['spd'].std()
                avg = ldx['spd'].mean()
                dur = m1 * d20dct[mzlst[i]] + m2* d20dct[mzlst[j]] + (1-m1-m2)* d20dct[mzlst[k]]
                ydct = dict(zip(yrcol,[grp,prd,mt1,mt2,mt3,m1,m2,dur,avg,sd]))
            else:
              pass
        yr = yr.append(ydct,ignore_index=True)
    print(mt1, sd, dur, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


r1.5 started at: 2019-05-24 18:17:59
r0 0.37265831870486793 2.144360781141865 2019-05-24 18:22:20
r1 0.3060964991975907 2.1651941144751983 2019-05-24 18:25:47
r3 0.30864949611886755 2.206860781141865 2019-05-24 18:28:29
r6 0.35651725631112896 2.0674247030276782 2019-05-24 18:30:30
r12 0.5066922929734583 2.412979395461974 2019-05-24 18:31:56
r24 0.8043277097992898 2.8179008330721373 2019-05-24 18:32:53
r36 0.9684408194023504 3.7287129980885236 2019-05-24 18:33:28
r48 0.9836023426059562 4.28872156228373 2019-05-24 18:33:45
r60 0.9836023426059562 4.28872156228373 2019-05-24 18:33:51
r1.5_1 started at: 2019-05-24 18:33:51
r0 0.4461658753518736 2.144360781141865 2019-05-24 18:38:09
r1 0.38984441566364325 2.1651941144751983 2019-05-24 18:41:36
r3 0.31465220093363677 2.206860781141865 2019-05-24 18:44:17
r6 0.24843666462410557 2.0674247030276782 2019-05-24 18:46:18
r12 0.3649864712585324 2.225405178779798 2019-05-24 18:47:44
r24 0.6820202400791034 2.327627256668273 2019-05-24 18:48:41
r36 0.9

In [30]:
yr.head()

grp  prod mt1 mt2  mt3   mx1  mx2       dur       spd        sd
0  TestProd2  r1.5  r0  r1   r3  0.25  0.1  0.085417 -1.342089  0.907300
1  TestProd2  r1.5  r0  r1   r6  0.20  0.2  0.158333 -1.462785  0.875889
2  TestProd2  r1.5  r0  r1  r12  0.00  0.5  0.289452 -1.552709  0.856415
3  TestProd2  r1.5  r0  r1  r24  0.05  0.5  0.479937 -1.792312  0.693689
4  TestProd2  r1.5  r0  r1  r36  0.00  0.6  0.620471 -1.938803  0.511192

In [31]:
yrn = yr.ix[yr.groupby(['prod']).apply(lambda x: x['sd'].idxmin())]
yrn

grp     prod mt1  mt2   mt3   mx1   mx2       dur       spd  \
1343  TestProd2      DDA  r0   r6  r120  0.00  0.05  4.086785 -5.693406   
59    TestProd2     r1.5  r1   r6  r120  0.40  0.15  1.984091 -2.809872   
291   TestProd2   r1.5_1  r6  r48  r120  0.55  0.10  1.832649 -2.734470   
1652  TestProd2      r12  r0   r1   r12  0.00  0.00  0.537237  0.000000   
1116  TestProd2    r12.5  r6  r48  r120  0.50  0.20  1.799808 -2.574428   
1271  TestProd2  r12.5_1  r6  r24   r48  0.45  0.05  1.133992 -2.132824   
417   TestProd2     r3.5  r3   r6  r120  0.35  0.20  2.023675 -2.803781   
621   TestProd2   r3.5_1  r6  r48  r120  0.55  0.25  1.480484 -2.457996   
1486  TestProd2       r6  r0   r1    r6  0.00  0.00  0.250000  0.000000   
683   TestProd2     r6.5  r0   r6  r120  0.00  0.55  2.067425 -2.773957   
949   TestProd2   r6.5_1  r6  r48   r60  0.55  0.35  1.054445 -2.127127   

            sd  
1343  0.604397  
59    0.300484  
291   0.243864  
1652  0.000000  
1116  0.240862  
1271  0.237223  
417   0.300824  
621   0.260405  
1486  0.000000  
683   0.304578  
949   0.294148

In [32]:
yr.to_csv('yldreg_3mat_'+grp+'_'+r0+'.csv')
yrn.to_csv('yldreg_3matbest_'+grp+'_'+r0+'.csv')

In [33]:
'yldreg_3matbest_'+grp+'_'+r0+'.csv'

'yldreg_3matbest_TestProd2_2001_2010.csv'

In [34]:
len(yr), len(yrn) # 1815, 11

(1815, 11)